# Assignment 2A, Part 2: Retrieval

Implement BM25 and LM retrieval.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
import pickle
import math
import itertools
import numpy as np

from collections import Counter
from tqdm import tqdm

from hashedindex import textparser
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

N_GRAMS = 1

In [19]:
QUERY_FILE = 'data/queries.txt'  # make sure the query file exists on this location
BM25_OUTPUT_FILE = 'data/bm25_singlefield.txt'  # output the BM25 ranking
LM_OUTPUT_FILE = 'data/lm_singlefield.txt'  # output the LM ranking

## Load index

In [7]:
def load_file(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

In [8]:
index = load_file('data/basic_index_new_idf.dat')

In [9]:
index['content_index'].terms()

[('los',),
 ('angeles',),
 ('every',),
 ('year',),
 ('millions',),
 ('americans',),
 ('pledge',),
 ('put',),
 ('financial',),
 ('house',),
 ('order',),
 ('say',),
 ('different',),
 ('theyll',),
 ('save',),
 ('invest',),
 ('even',),
 ('stick',),
 ('budget',),
 ('indeed',),
 ('second',),
 ('popular',),
 ('new',),
 ('resolution',),
 ('achieve',),
 ('goals',),
 ('according',),
 ('citibank',),
 ('first',),
 ('lose',),
 ('weight',),
 ('live',),
 ('healthfully',),
 ('well',),
 ('leave',),
 ('loss',),
 ('jenny',),
 ('craig',),
 ('hard',),
 ('steel',),
 ('dramatic',),
 ('decrease',),
 ('lifestyle',),
 ('dont',),
 ('panic',),
 ('many',),
 ('changes',),
 ('made',),
 ('without',),
 ('much',),
 ('suffering',),
 ('matter',),
 ('plugging',),
 ('money',),
 ('leaks',),
 ('life',),
 ('wont',),
 ('deflate',),
 ('italjoie',),
 ('de',),
 ('vivreend',),
 ('ital',),
 ('keep',),
 ('mind',),
 ('saving',),
 ('end',),
 ('race',),
 ('see',),
 ('one',),
 ('provide',),
 ('adequately',),
 ('future',),
 ('gives',),
 

In [10]:
index.keys()

dict_keys(['content_index', 'title_index', 'content_doc_length', 'title_doc_length', 'average_content_length', 'average_title_length', 'content_idf', 'title_idf', 'content_sum_tf', 'title_sum_tf', 'content_sum_length', 'title_sum_length', 'content_collection_probability', 'title_collection_probability'])

### Load the queries from the file

See the assignment description for the format of the query file [here](https://github.com/kbalog/uis-dat640-fall2019/tree/master/assignments/assignment-2a).

In [11]:
def load_queries(query_file):
    queries = {}
    with open(query_file, "r") as fin:
        for line in fin.readlines():
            qid, query = line.strip().split(" ", 1)
            queries[qid] = query
    return queries

In [12]:
queries = load_queries(QUERY_FILE)

## Retrieval models

In [13]:
class BM25():
    
    def __init__(self, index, k1 = 1.2, b = 0.75):
        self.k1 = k1
        self.b = b
        
        self.content_index = index['content_index']
        self.content_idf = index['content_idf']
        self.content_doc_length = index['content_doc_length']
        self.average_content_length = index['average_content_length']

    
    def rank_query(self, query):
        
        # tokenize with nltk tokenizer because it works well
        query = ' '.join([word for word in word_tokenize(query)])
        
        ranking = Counter()
        
        # tokenize a second time with hashedindex tokenizer to have tuple tokens
        for token in textparser.word_tokenize(query, stopwords=stopwords, ngrams=N_GRAMS):
            if token in self.content_index:
                
                documents = self.content_index.get_documents(token)
                #print('"{}" appears in {} documents'.format(token, len(documents)))   
                
                ranking += Counter(self.rank_term(token, documents))
            else:
                print(token, 'is not in the index')   
        
        return ranking.most_common(100)
        
    def rank_term(self, term, documents):
        # here we use a dictionnary because adding counters is a slow operation compared to dictionnary access
        document_scores = {}
        
        for doc in documents:
            tf = self.content_index.get_term_frequency(term, doc) # not normalized !
            smoothing = 1 - self.b + self.b * (self.content_doc_length[doc]/self.average_content_length)         
            score = ( ((1+self.k1)*tf) / (tf+self.k1*smoothing) ) * self.content_idf[term]
            document_scores[doc] = score

        return document_scores

In [22]:
class LM():
    
    def __init__(self, index, smoothing='jelinek', lambda_param=0.1, mu_param=1000):
        
        if smoothing == 'jelinek':
            self.lambda_param = lambda_param
        elif smoothing == 'dirichlet':
            self.mu_param = mu_param
        else:
            raise ValueError('smoothing should in [jelinek, dirichlet]')
            
        self.smoothing = smoothing
        
        self.content_index = index['content_index']
        self.content_doc_length = index['content_doc_length']
        self.content_sum_tf = index['content_sum_tf']
        self.content_sum_length = index['content_sum_length']
        self.content_collection_probability = index['content_collection_probability']
        
    
    def rank_query(self, query):
        query = ' '.join([word for word in word_tokenize(query)])
        
        # we can't use counter objects to track scores because it does not support negative addition
        ranking = {}
        
        for token in textparser.word_tokenize(query, stopwords=stopwords, ngrams=N_GRAMS):
            if token in self.content_index:
                documents = self.content_index.get_documents(token)
                #print('"{}" appears in {} documents'.format(token, len(documents)))
                
                if self.smoothing == 'jelinek':
                    ranking = self.merge_rankings(ranking, self.rank_term_jelinek(token, documents))
                elif self.smoothing == 'dirichlet':
                    ranking = self.merge_rankings(ranking, self.rank_term_dirichlet(token, documents))
                
            else:
                print(token, 'is not in the index')
                
        return sorted(ranking.items(), key=lambda x: x[1])[:100]
        
    def rank_term_jelinek(self, term, documents):
        document_scores = {}
        
        for doc in documents:
            ptd = self.content_index.get_term_frequency(term, doc, normalized=True)
            ptc = self.content_collection_probability[term]
            score = ((1 - self.lambda_param) * ptd) + (self.lambda_param * ptc)
            document_scores[doc] = math.log(score)
            
        return document_scores
    
    def rank_term_dirichlet(self, term, documents):
        document_scores = {}
        
        for doc in documents:
            tf = self.content_index.get_term_frequency(term, doc, normalized=True)
            ptc = self.content_collection_probability[term]
            score = (tf + self.mu_param*ptc) / (self.content_doc_length[doc] + self.mu_param)
            document_scores[doc] = math.log(score)
            
        return document_scores

    def merge_rankings(self, base_ranking, to_add):
        for doc, score in to_add.items():
            base_ranking[doc] = base_ranking.get(doc, 0) + score
        return base_ranking

### Perform retrieval

**TODO** Generate a ranking for each query and output the results to `OUTPUT_FILE`

See the assignment description for the format of the output file [here](https://github.com/kbalog/uis-dat640-fall2019/tree/master/assignments/assignment-2a).

In [24]:
bm25 = BM25(index, k1=1.2, b=0.1)

with open(BM25_OUTPUT_FILE, 'w') as f:
    f.write('QueryId,DocumentId\n')
    
    for q_id, query in queries.items():
        
        print("Ranking documents for [%s] '%s'" % (q_id, query))
        ranking = bm25.rank_query(query)
        f.writelines(['{},{}\n'.format(q_id, document[0], '\n') for document in ranking])
        f.flush()

Ranking documents for [303] 'Hubble Telescope Achievements'
Ranking documents for [307] 'New Hydroelectric Projects'
Ranking documents for [310] 'Radio Waves and Brain Cancer'
Ranking documents for [314] 'Marine Vegetation'
Ranking documents for [322] 'International Art Crime'
Ranking documents for [325] 'Cult Lifestyles'
Ranking documents for [330] 'Iran-Iraq Cooperation'
Ranking documents for [336] 'Black Bear Attacks'
Ranking documents for [341] 'Airport Security'
Ranking documents for [344] 'Abuses of E-Mail'
Ranking documents for [345] 'Overseas Tobacco Sales'
Ranking documents for [347] 'Wildlife Extinction'
Ranking documents for [353] 'Antarctica exploration'
Ranking documents for [354] 'journalist risks'
Ranking documents for [362] 'human smuggling'
Ranking documents for [363] 'transportation tunnel disasters'
Ranking documents for [367] 'piracy'
Ranking documents for [372] 'Native American casino'
Ranking documents for [374] 'Nobel prize winners'
Ranking documents for [375] 'h

In [23]:
lm = LM(index, smoothing='jelinek', lambda_param=1.0)

with open(LM_OUTPUT_FILE, 'w') as f:
    f.write('QueryId,DocumentId\n')
    
    for q_id, query in queries.items():
        
        print("Ranking documents for [%s] '%s'" % (q_id, query))
        ranking = lm.rank_query(query)
        f.writelines(['{},{}\n'.format(q_id, document[0], '\n') for document in ranking])
        f.flush()

Ranking documents for [303] 'Hubble Telescope Achievements'
Ranking documents for [307] 'New Hydroelectric Projects'
Ranking documents for [310] 'Radio Waves and Brain Cancer'
Ranking documents for [314] 'Marine Vegetation'
Ranking documents for [322] 'International Art Crime'
Ranking documents for [325] 'Cult Lifestyles'
Ranking documents for [330] 'Iran-Iraq Cooperation'
Ranking documents for [336] 'Black Bear Attacks'
Ranking documents for [341] 'Airport Security'
Ranking documents for [344] 'Abuses of E-Mail'
Ranking documents for [345] 'Overseas Tobacco Sales'
Ranking documents for [347] 'Wildlife Extinction'
Ranking documents for [353] 'Antarctica exploration'
Ranking documents for [354] 'journalist risks'
Ranking documents for [362] 'human smuggling'
Ranking documents for [363] 'transportation tunnel disasters'
Ranking documents for [367] 'piracy'
Ranking documents for [372] 'Native American casino'
Ranking documents for [374] 'Nobel prize winners'
Ranking documents for [375] 'h

In [15]:
def BM25_parameters_gridsearch(k_start=1.0, k_stop=2.1, k_step=0.1, b_start=0.0, b_stop=1.1, b_step=0.1):
    k_range = np.arange(k_start, k_stop, k_step)
    b_range = np.arange(b_start, b_stop, b_step)
    grid = itertools.product(k_range, b_range)
    
    for k, b in grid:
        print(k, b)
        model = BM25(index, k1=k, b=b)
        with open('data/gridsearch_bm25_k{:.3}_b{:.3}.txt'.format(k, b), 'w') as f:
            f.write('QueryId,DocumentId\n')

            for q_id, query in queries.items():
                ranking = model.rank_query(query)
                f.writelines(['{},{}\n'.format(q_id, document[0], '\n') for document in ranking])
                f.flush()

In [16]:
BM25_parameters_gridsearch()

1.0 0.0


KeyboardInterrupt: 

In [31]:
def LM_parameters_gridsearch(lambda_start = 0.0, lambda_stop = 1.1, lambda_step = 0.1, mu_start=1, mu_stop=10000, n_mu=5):
    smoothings = ['jelinek', 'dirichlet']
    lambda_range = np.arange(lambda_start, lambda_stop, lambda_step)
    mu_range = np.linspace(mu_start, mu_stop, n_mu)
    grid = [[smoothings[0], param] for param in lambda_range] + [[smoothings[1], param] for param in mu_range]
    
    for params in grid:
        print(params[0], params[1])
        model = LM(index, smoothing=params[0], lambda_param=params[1], mu_param=params[1])
        with open('data/gridsearch_lm_s{:.3}_param{:.3}.txt'.format(params[0], params[1]), 'w') as f:
            f.write('QueryId,DocumentId\n')

            for q_id, query in queries.items():
                ranking = model.rank_query(query)
                f.writelines(['{},{}\n'.format(q_id, document[0], '\n') for document in ranking])
                f.flush()

In [32]:
LM_parameters_gridsearch()

jelinek 0.0
jelinek 0.1
jelinek 0.2
jelinek 0.30000000000000004
jelinek 0.4
jelinek 0.5
jelinek 0.6000000000000001
jelinek 0.7000000000000001
jelinek 0.8
jelinek 0.9
jelinek 1.0
dirichlet 1.0
dirichlet 2500.75
dirichlet 5000.5
dirichlet 7500.25
dirichlet 10000.0


## Evaluation

Report on the evaluation results (using the [Evaluation notebook](1_Evaluation.ipynb)) here.

Describe the parameter settings used for the two methods:

The models parameters were decided using a grid search over the reasonable values, ie $k1 \in [1, 2]$, $b \in [0, 1]$ for BM25 and $smoothing \in [jelinek, dirichlet]$, $ \lambda \in [1, 2]$, $ \mu \in [10, 10000]$ for LM. Then, the values around the parameters yielding the best results were manually tested to check if some fine-tuning was possible. The parameters achieveing the highest mean accuracy precision were then tested on Kaggle.

Write the name of the corresponding output file in the table. These files should be pushed to your repository.


| **Method** | **Output file** | **Parameters** | **P@10** | **MAP** | **MRR** |
| -- | -- | -- | -- | -- | -- |
| BM25 | `data/bm25_singlefield.txt` | k1=1.2 b=0.1 | 0.2089 | 0.0793 | 0.4308 |
| LM | `data/lm_singlefield.txt` | smoothing=jelinek, lambda=1.0 | 0.0778 | 0.0206 | 0.1399 |
